In [ ]:
#  1. Building the Docker image from custom_container folder
!cd ../custom_container && docker build -t wine-quality-model .


In [ ]:
#  2. Running the training inside container
!docker run --rm -v $(pwd):/opt/ml wine-quality-model python /opt/ml/program/train.py


In [ ]:
#  3. Starting inference server in background (Flask app inside container)
!docker run -d -p 8080:8080 --name wine_server wine-quality-model


In [ ]:
#  4. Sending the test request to server
import requests, json

# Sample Example wine sample input (fixed acidity, volatile acidity, etc.)
sample_input = {
    "inputs": [[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4]]
}

response = requests.post(
    "http://localhost:8080/predict",
    data=json.dumps(sample_input),
    headers={"Content-Type": "application/json"},
)
print("Response:", response.json())

In [ ]:
#  5. Cleanup: stop and remove container
!docker stop wine_server
!docker rm wine_server
